In [1]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import time
import copy
from datetime import datetime
import sys
sys.path.append("..")
from training_loop import train_model

In [2]:
BATCH_SIZE = 20
LABELS = sorted([i[:-4] for i in os.listdir('icons')])
NUM_CLASSES = len(LABELS)

In [3]:
import numpy as np
import collections

class GaussianNoise(object):
    """
    Add gaussian noise to a numpy.ndarray (H x W x C)
    """
    def __init__(self, mean, sigma, random_state=np.random):
        self.sigma = sigma
        self.mean = mean
        self.random_state = random_state

    def __call__(self, image):
        row, col, ch = image.shape
        gauss = self.random_state.normal(self.mean, self.sigma, (row, col, ch))
        gauss = gauss.reshape(row, col, ch)
        image += torch.from_numpy(gauss).float()
        return image

In [4]:
class IconsDataset(Dataset):
    def __init__(self, directory, labels, transform=None):
        self.directory = directory
        self.files = os.listdir(directory)[:1000]
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        f = self.files[index]
        # image
        image = Image.open(os.path.join(self.directory, f))
        if self.transform:
            image = self.transform(image)
        
        # labels
        label = f.split('.')[0].split('_')[1]
        label = LABELS.index(label)
        return image, torch.tensor(label, dtype=torch.long)
    
    def input_size(self):
        return 100 * 100 * 3
    
    def __len__(self):
        return len(self.files)


def load_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        #GaussianNoise(0.01, 0.001),
    ])
    
    d = IconsDataset('images_single_224/', LABELS, transform=transform)
    size = len(d)
    train, validate = random_split(d, [int(size * 0.8), int(size * 0.2)])
    
    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)

    return d.input_size(), loader, validation_loader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512, NUM_CLASSES)
model = model.to(device)

input_size, loader, validation_loader = load_dataset()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=0.0001)
train_model('resnet18_single_4_batchsize20', model, {'train': loader, 'val': validation_loader}, criterion, optimizer, device, num_epochs=1000)

Epoch: 0, TL: 2.82135, VL: 3.02172, TA: 0.10500, VA: 0.05500
Epoch: 1, TL: 2.09282, VL: 2.08305, TA: 0.25375, VA: 0.20000
Epoch: 2, TL: 1.77519, VL: 1.87521, TA: 0.38375, VA: 0.25500
Epoch: 3, TL: 1.43635, VL: 1.81388, TA: 0.55625, VA: 0.32000
Epoch: 4, TL: 1.06223, VL: 2.18336, TA: 0.72250, VA: 0.26000
Epoch: 5, TL: 0.81416, VL: 3.99741, TA: 0.80625, VA: 0.21500
Epoch: 6, TL: 0.66446, VL: 4.67822, TA: 0.85625, VA: 0.13000
Epoch: 7, TL: 0.39619, VL: 3.48232, TA: 0.91000, VA: 0.13500
Epoch: 8, TL: 0.28634, VL: 4.65652, TA: 0.92750, VA: 0.13000
